In [0]:
%pip install -r requirements.txt

In [0]:
%restart_python

In [0]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from functools import partial
from src.data.ts_ops import resample

In [0]:
dbutils.widgets.text("file_date", "2023-01-01", "Data del file")
file_date = dbutils.widgets.get("file_date")

In [0]:
from config.paths import get_original_input_file_path, get_clean_input_file_path

INPUT_FILEPATH = get_original_input_file_path(file_date)
OUTPUT_FILEPATH = get_clean_input_file_path(file_date)


In [0]:
from src.data.columns import Columns

DATE_COL = Columns.DATE.value
AMOUNT_COL = Columns.AMOUNT.value
NAME_COL = Columns.NAME.value

In [0]:
data = pd.read_csv(INPUT_FILEPATH)

In [0]:
data = data.loc[data[NAME_COL] == 'series_1'].drop(columns=[NAME_COL])
data.columns

In [0]:
data[DATE_COL] = pd.to_datetime(data[DATE_COL], format='%d-%b-%y')
data = resample(data, col=DATE_COL, freq="B").fillna(0)
data.index = data[DATE_COL]
data = data.asfreq('B')

In [0]:
data[AMOUNT_COL].plot()
plt.show()

In [0]:
data

In [0]:
spark_df = spark.createDataFrame(data.reset_index(drop=True))
spark_df.write.format("delta").mode("overwrite").save(OUTPUT_FILEPATH)